In [2]:
# StraddleStrategy 实例运行示例
import sys
import os
from datetime import datetime, timedelta
from typing import Dict, Optional
from dataclasses import dataclass

# 添加vnpy路径
sys.path.append(r'e:\document\Code\vnpy\.vnpy_env\Lib\site-packages')

# 导入必要的模块
from vnpy.trader.object import TickData, OrderData, TradeData
from vnpy.trader.constant import Direction, Offset, Exchange
from vnpy_optionmaster.base import OptionData, UnderlyingData, ChainData, PortfolioData
from vnpy_optionmaster.straddle_strategy import StraddleStrategy, StraddleManager

print("模块导入成功")

模块导入成功


In [3]:
# 创建模拟的算法引擎
class MockOptionAlgoEngine:
    def __init__(self):
        self.orders = {}
        self.order_count = 0
        
    def send_order(self, strategy, vt_symbol: str, direction: Direction, 
                   offset: Offset, price: float, volume: int) -> str:
        self.order_count += 1
        vt_orderid = f"mock_order_{self.order_count}"
        
        order = OrderData(
            symbol=vt_symbol.split('.')[0],
            exchange=Exchange.CFFEX,
            orderid=str(self.order_count),
            direction=direction,
            offset=offset,
            price=price,
            volume=volume,
            gateway_name="mock"
        )
        
        self.orders[vt_orderid] = order
        print(f"发送委托: {vt_symbol} {direction.value} {offset.value} {volume}@{price}")
        
        # 模拟立即成交
        self.simulate_trade(strategy, order)
        
        return vt_orderid
    
    def simulate_trade(self, strategy, order: OrderData):
        """模拟成交"""
        trade = TradeData(
            symbol=order.symbol,
            exchange=order.exchange,
            orderid=order.orderid,
            tradeid=f"trade_{order.orderid}",
            direction=order.direction,
            offset=order.offset,
            price=order.price,
            volume=order.volume,
            datetime=datetime.now(),
            gateway_name=order.gateway_name
        )
        trade.vt_symbol = f"{trade.symbol}.{trade.exchange.value}"
        trade.size = 10000  # 合约乘数
        
        # 通知策略成交
        strategy.on_trade(trade)
    
    def write_log(self, msg: str):
        print(f"Engine Log: {msg}")

# 创建算法引擎实例
algo_engine = MockOptionAlgoEngine()
print("算法引擎创建完成")

算法引擎创建完成


In [4]:
# 创建模拟的期权和标的数据
def create_mock_tick(symbol: str, last_price: float, bid_price: float, ask_price: float) -> TickData:
    """创建模拟行情数据"""
    tick = TickData(
        symbol=symbol.split('.')[0],
        exchange=Exchange.CFFEX,
        datetime=datetime.now(),
        last_price=last_price,
        bid_price_1=bid_price,
        ask_price_1=ask_price,
        bid_volume_1=10,
        ask_volume_1=10,
        gateway_name="mock"
    )
    tick.vt_symbol = symbol
    return tick

# 创建标的数据
from vnpy.trader.object import ContractData
contract = ContractData(
    symbol="510050",
    exchange=Exchange.SSE,
    name="华夏上证50ETF",
    product="ETF",
    size=10000,
    pricetick=0.001,
    gateway_name="mock"
)
underlying = UnderlyingData(contract)
underlying.vt_symbol = "510050.SSE"
underlying.price = 3.000
underlying.theo_delta = 1.0

# 创建看涨期权合约数据
call_contract = ContractData(
    symbol="10004186",
    exchange=Exchange.CFFEX,
    name="华夏上证50ETF看涨期权",
    product="OPTION",
    size=10000,
    pricetick=0.001,
    gateway_name="mock"
)
# 设置期权到期日期
call_contract.option_expiry = datetime.now() + timedelta(days=30)
call_contract.option_strike = 3.000
call_contract.option_type = "CALL"

# 创建看涨期权数据
call_option = OptionData(call_contract)
call_option.vt_symbol = "10004186.CFFEX"  # 看涨期权
call_option.strike_price = 3.000
call_option.days_to_expiry = 30
call_option.net_pos = 0
call_option.pos_delta = 0
call_option.pos_gamma = 0
call_option.pos_vega = 0
call_option.size = 10000
call_option.tick = create_mock_tick("10004186.CFFEX", 0.155, 0.153, 0.157)

# 创建看跌期权合约数据
put_contract = ContractData(
    symbol="10004187",
    exchange=Exchange.CFFEX,
    name="华夏上证50ETF看跌期权",
    product="OPTION",
    size=10000,
    pricetick=0.001,
    gateway_name="mock"
)
# 设置期权到期日期
put_contract.option_expiry = datetime.now() + timedelta(days=30)
put_contract.option_strike = 3.000
put_contract.option_type = "PUT"
# 创建看跌期权数据
put_option = OptionData(put_contract)
put_option.vt_symbol = "10004187.CFFEX"  # 看跌期权
put_option.strike_price = 3.000
put_option.days_to_expiry = 30
put_option.net_pos = 0
put_option.pos_delta = 0
put_option.pos_gamma = 0
put_option.pos_vega = 0
put_option.size = 10000
put_option.tick = create_mock_tick("10004187.CFFEX", 0.145, 0.143, 0.147)

# 创建期权链数据
# 为ChainData提供必需的参数
chain_symbol = "510050_O"  # 期权链标识符
event_engine = None  # 如果不需要事件引擎可以传None

chain = ChainData(chain_symbol, event_engine)
chain.underlying = underlying
chain.calls = {"3000": call_option}
chain.puts = {"3000": put_option}

print("模拟数据创建完成")
print(f"看涨期权: {call_option.vt_symbol}, 价格: {call_option.tick.last_price}")
print(f"看跌期权: {put_option.vt_symbol}, 价格: {put_option.tick.last_price}")

模拟数据创建完成
看涨期权: 10004186.CFFEX, 价格: 0.155
看跌期权: 10004187.CFFEX, 价格: 0.145


In [ ]:
# 创建并配置跨式策略
strategy = StraddleStrategy(
    strategy_name="ETF_Straddle_3000",
    algo_engine=algo_engine,
    chain=chain,
    strike_index="3000"
)

# 策略参数配置
strategy_params = {
    "target_volume": 2,           # 目标持仓2手
    "max_loss": 5000.0,          # 最大亏损5000元
    "profit_target": 2000.0,     # 盈利目标2000元
    "days_to_close": 5,          # 距离到期5天平仓
    "delta_limit": 100.0,        # Delta限制
    "gamma_limit": 50.0,         # Gamma限制
    "vega_limit": 100.0,         # Vega限制
    "hedge_active": True,        # 启用动态对冲
    "hedge_delta_threshold": 50.0 # Delta对冲阈值
}

print("策略参数配置:")
for key, value in strategy_params.items():
    print(f"  {key}: {value}")

策略参数配置:
  target_volume: 2
  max_loss: 5000.0
  profit_target: 2000.0
  days_to_close: 5
  delta_limit: 100.0
  gamma_limit: 50.0
  vega_limit: 100.0
  hedge_active: True
  hedge_delta_threshold: 50.0


In [7]:
# 确保期权对象有underlying属性
call_option.underlying = underlying
put_option.underlying = underlying

# 启动策略
print("\n=== 启动跨式策略 ===")
success = strategy.start_strategy(strategy_params)

if success:
    print("策略启动成功!")
    
    # 显示策略状态
    status = strategy.get_status()
    print("\n策略状态:")
    for key, value in status.items():
        print(f"  {key}: {value}")
else:
    print("策略启动失败!")


=== 启动跨式策略 ===
发送委托: 10004186.CFFEX 空 开 2@0.153
11:05:21 [ETF_Straddle_3000] 策略成交：10004186.CFFEX Direction.SHORT 2@0.153
Engine Log: [ETF_Straddle_3000] 策略成交：10004186.CFFEX Direction.SHORT 2@0.153
11:05:21 [ETF_Straddle_3000] 发送委托：10004186.CFFEX Direction.SHORT Offset.OPEN 2@0.153 [mock_order_1]
Engine Log: [ETF_Straddle_3000] 发送委托：10004186.CFFEX Direction.SHORT Offset.OPEN 2@0.153 [mock_order_1]
发送委托: 10004187.CFFEX 空 开 2@0.143
11:05:21 [ETF_Straddle_3000] 策略成交：10004187.CFFEX Direction.SHORT 2@0.143
Engine Log: [ETF_Straddle_3000] 策略成交：10004187.CFFEX Direction.SHORT 2@0.143
11:05:21 [ETF_Straddle_3000] 发送委托：10004187.CFFEX Direction.SHORT Offset.OPEN 2@0.143 [mock_order_2]
Engine Log: [ETF_Straddle_3000] 发送委托：10004187.CFFEX Direction.SHORT Offset.OPEN 2@0.143 [mock_order_2]
11:05:21 [ETF_Straddle_3000] 跨式策略启动：10004186.CFFEX & 10004187.CFFEX
Engine Log: [ETF_Straddle_3000] 跨式策略启动：10004186.CFFEX & 10004187.CFFEX
策略启动成功!

策略状态:
  strategy_name: ETF_Straddle_3000
  active: True
  strike_i

In [8]:
# 模拟市场行情变化
print("\n=== 模拟市场行情变化 ===")

# 场景1: 市场平稳，期权价格下跌（有利于卖方）
print("\n场景1: 市场平稳，期权价格下跌")
call_option.tick = create_mock_tick("10004186.CFFEX", 0.120, 0.118, 0.122)
put_option.tick = create_mock_tick("10004187.CFFEX", 0.115, 0.113, 0.117)

# 更新持仓（模拟已经卖出期权）
call_option.net_pos = -2
put_option.net_pos = -2

# 触发行情更新
strategy.on_tick(call_option.tick)
strategy.on_tick(put_option.tick)

current_pnl = strategy.calculate_pnl()
print(f"当前盈亏: {current_pnl:.2f} 元")

print(f"看涨期权价格: {call_option.tick.last_price} (入场价格: {strategy.entry_call_price})")
print(f"看跌期权价格: {put_option.tick.last_price} (入场价格: {strategy.entry_put_price})")


=== 模拟市场行情变化 ===

场景1: 市场平稳，期权价格下跌
当前盈亏: 1220.00 元
看涨期权价格: 0.12 (入场价格: 0.153)
看跌期权价格: 0.115 (入场价格: 0.143)


In [9]:
# 场景2: 市场大幅波动，期权价格上涨（不利于卖方）
print("\n场景2: 市场大幅波动，期权价格上涨")
call_option.tick = create_mock_tick("10004186.CFFEX", 0.200, 0.198, 0.202)
put_option.tick = create_mock_tick("10004187.CFFEX", 0.180, 0.178, 0.182)

# 触发行情更新
strategy.on_tick(call_option.tick)
strategy.on_tick(put_option.tick)

current_pnl = strategy.calculate_pnl()
print(f"当前盈亏: {current_pnl:.2f} 元")

print(f"看涨期权价格: {call_option.tick.last_price}")
print(f"看跌期权价格: {put_option.tick.last_price}")

# 检查是否触发止损
if current_pnl <= -strategy.max_loss:
    print("⚠️ 触发最大亏损限制!")


场景2: 市场大幅波动，期权价格上涨
当前盈亏: -1680.00 元
看涨期权价格: 0.2
看跌期权价格: 0.18


In [10]:
# 场景3: 临近到期，触发时间止损
print("\n场景3: 临近到期，触发时间止损")
call_option.days_to_expiry = 3  # 距离到期3天
put_option.days_to_expiry = 3

strategy.on_tick(call_option.tick)

print(f"距离到期天数: {call_option.days_to_expiry}")
print(f"时间止损设置: {strategy.days_to_close} 天")


场景3: 临近到期，触发时间止损
11:06:06 [ETF_Straddle_3000] 时间止损：距离到期日过近，开始平仓
Engine Log: [ETF_Straddle_3000] 时间止损：距离到期日过近，开始平仓
发送委托: 10004186.CFFEX 多 平 2@0.202
11:06:06 [ETF_Straddle_3000] 策略成交：10004186.CFFEX Direction.LONG 2@0.202
Engine Log: [ETF_Straddle_3000] 策略成交：10004186.CFFEX Direction.LONG 2@0.202
11:06:06 [ETF_Straddle_3000] 发送委托：10004186.CFFEX Direction.LONG Offset.CLOSE 2@0.202 [mock_order_3]
Engine Log: [ETF_Straddle_3000] 发送委托：10004186.CFFEX Direction.LONG Offset.CLOSE 2@0.202 [mock_order_3]
发送委托: 10004187.CFFEX 多 平 2@0.182
11:06:06 [ETF_Straddle_3000] 策略成交：10004187.CFFEX Direction.LONG 2@0.182
Engine Log: [ETF_Straddle_3000] 策略成交：10004187.CFFEX Direction.LONG 2@0.182
11:06:06 [ETF_Straddle_3000] 发送委托：10004187.CFFEX Direction.LONG Offset.CLOSE 2@0.182 [mock_order_4]
Engine Log: [ETF_Straddle_3000] 发送委托：10004187.CFFEX Direction.LONG Offset.CLOSE 2@0.182 [mock_order_4]
距离到期天数: 3
时间止损设置: 5 天


In [13]:
# 使用策略管理器管理多个策略
print("\n=== 策略管理器演示 ===")

manager = StraddleManager(algo_engine)

# 创建多个不同执行价格的跨式策略
strike_prices = ["2950", "3000", "3050"]

for strike in strike_prices:
    # 为每个执行价格创建模拟期权数据
    call = OptionData(call_contract)
    call.vt_symbol = f"call_{strike}.CFFEX"
    call.strike_price = float(strike) / 100
    call.days_to_expiry = 30
    call.net_pos = 0
    call.size = 10000
    call.tick = create_mock_tick(call.vt_symbol, 0.100, 0.098, 0.102)
    
    put = OptionData(put_contract)
    put.vt_symbol = f"put_{strike}.CFFEX"
    put.strike_price = float(strike) / 100
    put.days_to_expiry = 30
    put.net_pos = 0
    put.size = 10000
    put.tick = create_mock_tick(put.vt_symbol, 0.095, 0.093, 0.097)
    
    # 创建链数据
    chain = ChainData(chain_symbol, event_engine)
    chain.underlying = underlying
    chain.calls = {strike: call}
    chain.puts = {strike: put}
    
    # 创建策略
    strategy_name = f"Straddle_{strike}"
    strategy = manager.create_strategy(strategy_name, chain, strike)
    
    print(f"创建策略: {strategy_name}, 执行价格: {strike}")

print(f"\n总共创建了 {len(manager.get_all_strategies())} 个策略")

# 显示所有策略状态
for strategy in manager.get_all_strategies():
    status = strategy.get_status()
    print(f"策略 {status['strategy_name']}: 活跃={status['active']}, 执行价格={status['strike_index']}")


=== 策略管理器演示 ===
创建策略: Straddle_2950, 执行价格: 2950
创建策略: Straddle_3000, 执行价格: 3000
创建策略: Straddle_3050, 执行价格: 3050

总共创建了 3 个策略
策略 Straddle_2950: 活跃=False, 执行价格=2950
策略 Straddle_3000: 活跃=False, 执行价格=3000
策略 Straddle_3050: 活跃=False, 执行价格=3050


In [14]:
# 策略性能分析和总结
print("\n=== 策略性能分析 ===")

def analyze_strategy_performance(strategy: StraddleStrategy):
    """分析策略性能"""
    status = strategy.get_status()
    
    print(f"\n策略名称: {status['strategy_name']}")
    print(f"执行价格: {status['strike_index']}")
    print(f"看涨期权: {status['call_symbol']}")
    print(f"看跌期权: {status['put_symbol']}")
    print(f"看涨持仓: {status['call_pos']}")
    print(f"看跌持仓: {status['put_pos']}")
    print(f"总权利金收入: {status['total_premium']:.2f}")
    print(f"当前盈亏: {status['current_pnl']:.2f}")
    print(f"距离到期: {status['days_to_expiry']} 天")
    print(f"入场时间: {status['entry_time']}")
    
    # 计算收益率
    if status['total_premium'] > 0:
        return_rate = (status['current_pnl'] / status['total_premium']) * 100
        print(f"收益率: {return_rate:.2f}%")
    
    return status

# 分析第一个策略
if manager.get_all_strategies():
    first_strategy = manager.get_all_strategies()[0]
    analyze_strategy_performance(first_strategy)

print("\n=== 跨式策略运行示例完成 ===")
print("该示例展示了:")
print("1. 如何创建和配置跨式策略")
print("2. 如何启动策略并进行交易")
print("3. 如何模拟不同市场场景")
print("4. 如何使用策略管理器管理多个策略")
print("5. 如何分析策略性能")


=== 策略性能分析 ===

策略名称: Straddle_2950
执行价格: 2950
看涨期权: call_2950.CFFEX
看跌期权: put_2950.CFFEX
看涨持仓: 0
看跌持仓: 0
总权利金收入: 0.00
当前盈亏: 0.00
距离到期: 30 天
入场时间: 

=== 跨式策略运行示例完成 ===
该示例展示了:
1. 如何创建和配置跨式策略
2. 如何启动策略并进行交易
3. 如何模拟不同市场场景
4. 如何使用策略管理器管理多个策略
5. 如何分析策略性能
